In [1]:
from notebooks.lambda_imports import *
%matplotlib inline
plot_with_dark_style()

plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

In [2]:
session_name='AvellanedaQNotebook'
load_notebook_session(session_name)

notebook session file found AvellanedaQNotebook


# Properties

## Algorithm

In [3]:
parameters_default = {
    # Q
    "skewPricePctAction": [0.0],
    "riskAversionAction": [0.9, 0.5],
    "windowsTickAction": [10,15],
    "minPrivateState": (-0.001),
    "maxPrivateState": (0.001),
    "numberDecimalsPrivateState": (4),
    "horizonTicksPrivateState": (1),
    "horizonMinMsTick": (10),
    "scoreEnum": ScoreEnum.total_pnl,
    "timeHorizonSeconds": (5),
    "epsilon": (0.2),
    # Avellaneda default
    "risk_aversion": (0.9),
    "position_multiplier": (100),
    "window_tick": (10),
    "minutes_change_k": (10),
    "quantity": (0.0001),
    "k_default": (0.00769),
    "spread_multiplier": (5.0),
    "first_hour": (7),
    "last_hour": (19),
}
#for optimization
parameters_min={
    "risk_aversion": (0.1),
    "k_default": (0.001),
    "window_tick": (5)
}
parameters_max={
    "risk_aversion": (0.9),
    "k_default": (0.01),
    "window_tick": (25)
}


algorithm_info='q_as_notebook'
algorithm = AvellanedaQ(algorithm_info=algorithm_info,parameters=parameters_default)

## Backtest

In [4]:
instrument_pk='btcusdt_binance'

start_date_pt=datetime.datetime(year=2020, day=8, month=12)
end_date_pt=datetime.datetime(year=2020, day=8, month=12)

start_date_train=end_date_pt+datetime.timedelta(days=1)
end_date_train=start_date_train

start_date_test=start_date_train+datetime.timedelta(days=1)
end_date_test=start_date_test

## Genetic algorithm configuration

In [5]:
ga_configuration = GAConfiguration
ga_configuration.population = 15
ga_configuration_generations=3
ga_configuration_simultaneous=2

ga_configuration.decay=1/float(ga_configuration_generations)

# Parameter tuning

In [ ]:
best_param_dict, summary_df =algorithm.parameter_tuning(
    instrument_pk=instrument_pk,
    start_date=start_date_pt,
    end_date=end_date_pt,
    parameters_min=parameters_min,
    parameters_max=parameters_max,
    generations=ga_configuration_generations,
    ga_configuration=ga_configuration,
	max_simultaneous=ga_configuration_simultaneous
)

launching AvellanedaQ_ga_0
launching AvellanedaQ_ga_1
pwd=D:\javif\Coding\cryptotradingdesk\python_lambda\notebooks
pwd=D:\javif\Coding\cryptotradingdesk\python_lambda\notebooks


In [ ]:
summary_df

In [ ]:
best_by_generation=summary_df.groupby('generation').max('score')
best_by_generation['score'].plot()


In [ ]:
best_param_dict

# Train

In [ ]:
algorithm.parameters = best_param_dict
algorithm.algorithm_info=algorithm_info

In [ ]:
iterations_train=ga_configuration_generations
algos_per_iteration=ga_configuration.population
train_simultaneous=ga_configuration_simultaneous

output_train=algorithm.train(
        instrument_pk=instrument_pk,
        start_date=start_date_train,
        end_date=end_date_train,
        iterations=iterations_train,
        algos_per_iteration=algos_per_iteration,
        simultaneous_algos=train_simultaneous,
    )

In [ ]:
output_pnl=[]
legends=[]
for iteration in output_train:
	best_algorithm=None
	best_pnl=-99999
	for algorithm_name in iteration.keys():
		pnl=iteration[algorithm_name]['historicalTotalPnl'].iloc[-1]
		if pnl>best_pnl:
			best_pnl=pnl
			best_algorithm=algorithm_name
	legends.append('iteration %d'%len(output_pnl))
	output_pnl.append(best_pnl)
	
	
	plt.subplot(211)
	plt.title('pnl all')
	iteration[best_algorithm]['historicalTotalPnl'].plot()
	plt.legend(legends)
	
	plt.subplot(212)
	plt.title('pnl evolution')
	pd.Series(output_pnl).plot()
	plt.tight_layout()

# Test

In [ ]:
output_test = algorithm.test(
        instrument_pk=instrument_pk,
        start_date=start_date_test,
        end_date=end_date_test,
    )


In [ ]:
name_output=algorithm_info
backtest_result_test=output_test[name_output]
backtest_result_test.tail()

In [ ]:
backtest_result_test['historicalTotalPnl'].plot()
plt.title('backtest test')

In [ ]:
save_notebook_session(session_name)